<a href="https://colab.research.google.com/github/DagmarMpheio/fruit_ripeness_yolo8/blob/main/Fruit_Ripeness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8

# **DETENÇÃO DO ESTADO DE MATU**

# Instalar Utralytics

In [ ]:
!nvidia-smi

Tue Jan 16 21:31:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
import threading
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()
!yolo checks

Ultralytics YOLOv8.1.2 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)

OS                  Linux-6.1.58+-x86_64-with-glibc2.35
Environment         Colab
Python              3.10.12
Install             pip
RAM                 12.67 GB
CPU                 Intel Xeon 2.00GHz
CUDA                12.1

matplotlib          ✅ 3.7.1>=3.3.0
numpy               ✅ 1.23.5>=1.22.2
opencv-python       ✅ 4.8.0.76>=4.6.0
pillow              ✅ 9.4.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.1.0+cu121>=1.8.0
torchvision         ✅ 0.16.0+cu121>=0.9.0
tqdm                ✅ 4.66.1>=4.64.0
psutil              ✅ 5.9.5
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1
pandas              ✅ 1.5.3>=1.1.4
seaborn             ✅ 0.13.1>=0.11.0


# Criar a divisão de dados para treino e validação

In [ ]:
# importar as bibliotecas necessarios
import os
import shutil
import threading
import time
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.6
    Uninstalling tqdm-4.66.6:
      Successfully uninstalled tqdm-4.66.6


In [ ]:
# conectando ao google drive

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#pip install -r requirements.txt

In [26]:
train_path_img="./yolov8_detecao_maturacao_frutas/images/train"
train_path_label="./yolov8_detecao_maturacao_frutas/labels/train"
val_path_img="./yolov8_detecao_maturacao_frutas/images/val"
val_path_label="./yolov8_detecao_maturacao_frutas/labels/val"
test_path="./yolov8_detecao_maturacao_frutas/test"

In [40]:
'''
Dividir o dataset em treino e teste e cria o train.txt e test.txt com
o respectivo caminho das imagens em cada pasta
'''

def train_test_split(path, neg_path=None, split = 0.2):
  print("------ PROCESSO INICIADO ------")

  files = list(set([name[:-4] for name in os.listdir(path)])) ## remover nomes duplicados, isto e, contar apenas o numero de imagens

  print(f"--- Esta pasta tem no total {len(files)} imagens---")
  random.seed(42)
  random.shuffle(files)

  test_size = int(len(files) * split)
  train_size = len(files) - test_size

  ## criar os directorios necessarios

  os.makedirs(train_path_img, exist_ok = True)
  os.makedirs(train_path_label, exist_ok = True)
  os.makedirs(val_path_img, exist_ok = True)
  os.makedirs(val_path_label, exist_ok = True)


  ### ----------- copiar as imagens na pasta de treino
  for filex in tqdm(files[:train_size]):
    if filex == 'classes':
      continue
    shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
    shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')


  print(f"------ Dados de Treino criado com 80% de divisao das {len(files[:train_size])} imagens -------")

  if neg_path:
    neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## remover nomes duplicados, isto e, contar apenas o numero de imagens
    for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

    print(f"------ Total  {len(neg_images)} imagens negativas adicionadas nos dados de treino -------")

    print(f"------ TOTAL dados de treino criado com um total de {len(files[:train_size]) + len(neg_images)} imagens -------")


  ### copiar as imagens para a pasta de validacao
  for filex in tqdm(files[train_size:]):
    if filex == 'classes':
      continue
    # print("running")
    shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
    shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Dados de teste criado com um total de {len(files[train_size:])} imagens ----------")

    print("------ TAREFA CONCLUIDA -------")


### for label_tag
train_test_split('/content/drive/MyDrive/dataset-frutas/') ### sem imagens negativas

------ PROCESSO INICIADO ------
--- Esta pasta tem no total 17 imagens---


  0%|          | 0/14 [00:00<?, ?it/s]

------ Dados de Treino criado com 80% de divisao das 14 imagens -------


  0%|          | 0/3 [00:00<?, ?it/s]

------ Dados de teste criado com um total de 3 imagens ----------
------ TAREFA CONCLUIDA -------
------ Dados de teste criado com um total de 3 imagens ----------
------ TAREFA CONCLUIDA -------
------ Dados de teste criado com um total de 3 imagens ----------
------ TAREFA CONCLUIDA -------


In [29]:
import ultralytics

ultralytics.checks()

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/112.6 GB disk)


# Treinar o modelo

In [41]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/drive/MyDrive/yolov8/dataset-frutas.yaml epochs=20 patience=20 batch=16 imgsz=640 project=/content/drive/MyDrive/yolov8/training_results name='fruits'

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/yolov8/dataset-frutas.yaml, epochs=20, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=fruits, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

### Copiar algumas images para teste

In [42]:
!cp -av '/content/yolov8_detecao_maturacao_frutas/images/val/' '/content/drive/MyDrive/yolov8/test_images_fruits/'

'/content/yolov8_detecao_maturacao_frutas/images/val/' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val'
'/content/yolov8_detecao_maturacao_frutas/images/val/images2.jpg' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val/images2.jpg'
'/content/yolov8_detecao_maturacao_frutas/images/val/images.jpg' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val/images.jpg'
'/content/yolov8_detecao_maturacao_frutas/images/val/moa3ufu.jpg' -> '/content/drive/MyDrive/yolov8/test_images_fruits/val/moa3ufu.jpg'


# Outra avaliação do modelo

In [45]:
!yolo task=detect mode=val model='/content/drive/MyDrive/yolov8/training_results/fruits/weights/best.pt' data=/content/drive/MyDrive/yolov8/dataset-frutas.yaml

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,853,077 parameters, 0 gradients, 78.8 GFLOPs
val: Scanning /content/yolov8_detecao_maturacao_frutas/labels/val.cache... 3 images, 0 backgrounds, 0 corrupt: 100% 3/3 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:01<00:00,  1.02s/it]
                   all          3         10      0.599       0.42      0.503      0.431
         Morango Verde          1          1          1          0          0          0
   Morango Semi Maduro          1          1          1          0      0.497      0.348
        Morango Maduro          1          1     0.0863      0.518      0.166      0.116
           Maçã Madura          2          4       0.38          1      0.697      0.464
            Maçã Podre          2          2          1          0      0.663      0.663
      Maçã Semi Madura          1       

In [46]:
#copiar diretorios
!cp -r /content/runs/detect/val /content/drive/MyDrive/yolov8/val-20-epocas-batch-16-patience-20-modelo-m

# Inferência

In [47]:
!yolo task=detect mode=predict model='/content/drive/MyDrive/yolov8/training_results/fruits/weights/best.pt' conf=0.55 save=True source=/content/drive/MyDrive/yolov8/test_images_fruits

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,853,077 parameters, 0 gradients, 78.8 GFLOPs

image 1/3 /content/drive/MyDrive/yolov8/test_images_fruits/images.jpg: 640x416 (no detections), 122.8ms
image 2/3 /content/drive/MyDrive/yolov8/test_images_fruits/images2.jpg: 576x640 (no detections), 77.1ms
image 3/3 /content/drive/MyDrive/yolov8/test_images_fruits/moa3ufu.jpg: 512x640 1 Maçã Semi Madura, 74.8ms
Speed: 5.0ms preprocess, 91.6ms inference, 319.9ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [49]:
#copiar diretorios
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output-20-epocas-batch-16-patience-20-modelo-m

# Inferência em videos

In [ ]:
!yolo task=detect mode=predict model='/content/drive/MyDrive/yolov8/training_results/fruits/weights/best.pt' conf=0.55 save=True source=/content/drive/MyDrive/yolov8/test_videos_frutas

Streaming output truncated to the last 5000 lines.
video 3/4 (5153/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.2ms
video 3/4 (5154/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.3ms
video 3/4 (5155/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.5ms
video 3/4 (5156/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.5ms
video 3/4 (5157/7704) /content/drive/MyDrive/yolov8/test_videos/OLHA O QUE O Policial fez com motoqueiro sem capacete-Por essa ninguém esperava.mp4: 384x640 1 C/Cap, 1 S/Cap, 13.7ms
video 3/4 (5158/7704) /content/dri

In [ ]:
#copiar diretorios com resultados de inferencia em videos
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/output-3-epocas-batch-16-patience-10-modelo-m

In [39]:
#exluir diretorios nao vazios
!rm -r /content/runs

In [ ]:
!python --version